In [6]:
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as plt
import time
from tqdm.auto import tqdm
import numpy as np

import requests
from datetime import datetime
from ensembledata.api import EDClient

# Retrieve Comments

In [7]:
client = EDClient("tHMlAfSzgpDhxmFA")

In [8]:
df = pd.read_csv('idalia_posts.csv')

len(df)

33

In [9]:
# Convert time to numeric if needed (e.g., integer indices for regression)
x = np.arange(len(df["createTime"]))
y = df["commentCount"]

# Fit a polynomial regression (high degree for smooth curve)
coeffs = np.polyfit(x, y, deg=2)
poly = np.poly1d(coeffs)

# Generate smooth values for the regression line
x_smooth = np.linspace(x.min(), x.max(), 200)
y_smooth = poly(x_smooth)
plt.figure(figsize=(10, 6))

# Bar chart
plt.bar(df["createTime"], y, color='b', alpha=0.7, label='Comment Count (Bar)')

# Plot the polynomial regression line
plt.plot(x_smooth, y_smooth, color='r', linewidth=2, label='Trend Line (Poly. Regression)')

# Customize the plot
plt.title('Shares per Time, Idalia', fontsize=16)
plt.xlabel('Dates', fontsize=14)
plt.ylabel('Number of Shares', fontsize=14)
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.xticks(ticks=x, rotation=45)
plt.legend()  # Add legend

# Show the plot
plt.tight_layout()
plt.show()


In [10]:
# WE DO THIS TOGETHER
## YOU HAVE USED 20 UNITS IN ORDER TO RETRIEVE POSTS (OUT OF 50)

## COMMENTS TAKE A LOT OF UNITS TO RETRIEVE
## EACH DATASET ITERATES THROUGH POSTS, SO EACH HURRICANE HAS DIFFERENT COSTS

## DO THE MATH TO USE LESS THAN 50 UNITS OR IT WILL NOT RUN AND EMPTY YOUR TOKENS!
## SEE THE LAST PRINTED DATASET TO CHECK HOW MANY ROWS YOU HAVE TO ITERATE THROUGH
#---

comments_to_retrieve = 30

comments = list()
for i in range(len(df)):
    aweme_id = df["id"][i]
    cursor = None
    for _ in range(int(comments_to_retrieve / 30)):
        result = client.tiktok.post_comments(
            aweme_id=aweme_id,
            cursor=cursor
        )
        comments.extend(result.data["comments"])
        cursor = result.data.get("nextCursor")
        if cursor is None:
            break
            
# convert json to dataframe
df = pd.json_normalize(comments)

# extract relevant info
df = df[['aweme_id', 'cid', 'create_time', 'text', 'reply_comment_total']]

# format time to date in string format
df["create_time"] = [str(datetime.fromtimestamp(int(i)).date())
                     for i in df["create_time"]]

# order by number of replies
df = df.sort_values(by=['reply_comment_total'], ascending=False).reset_index(drop=True)

# save
df_comments = df
df_comments.to_csv('idalia_comments.csv', index=False)
df_comments

aweme_id                  cid create_time  \
0    7273191672803101994  7273223272096285486  2023-08-30   
1    7273191672803101994  7273230716459189035  2023-08-30   
2    7272535297378766123  7273434196776174366  2023-08-31   
3    7272071106490600747  7272137828543791903  2023-08-28   
4    7273191672803101994  7273225367933993770  2023-08-30   
..                   ...                  ...         ...   
454  7274413941835681070  7274417268406666027  2023-09-03   
455  7274413941835681070  7274417389960741674  2023-09-03   
456  7273620466760305963  7273675875563635498  2023-09-01   
457  7273620466760305963  7285516669102916357  2023-10-03   
458  7293469706407677214  7293565148333359915  2023-10-24   

                                                  text  reply_comment_total  
0    The second hand embarrassment for people that ...                  153  
1    I'm not sure why they bother to allow Fox news...                  115  
2    why are you so calm 😭 “and no insurance doesn’...                   89  
3    New To Florida. Just nine months. First experi...                   81  
4    Do people think all the rest of the people who...                   78  
..                                                 ...                  ...  
454  True leader. No trash talking just take care o...                    0  
455                              Dsansits is a coward💙                    0  
456                  @msmisfit I think I’m in love 🥰🥰🥰                    0  
457  Easier to throw weight around with a counterba...                    0  
458                       Heard in Vermont! Prayers ❤️                    0  

[459 rows x 5 columns]